<a href="https://colab.research.google.com/github/elizabethavargas/Dataset-Description-Generation/blob/main/testing_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Testing Prompts
The paper used GPT 4o-mini and LLaMA-3.1-8B-Instruct. However, GPT 4o-mini is no longer available so we can switch to 5-mini?

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git@31b667b54139962832ea2de890383eed14a0a17d"

  Cloning https://github.com/unslothai/unsloth.git (to revision 31b667b54139962832ea2de890383eed14a0a17d) to /tmp/pip-install-19cliqa3/unsloth_e5017ab745c9486f882c802aebd8fe6b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-19cliqa3/unsloth_e5017ab745c9486f882c802aebd8fe6b
  Running command git rev-parse -q --verify 'sha^31b667b54139962832ea2de890383eed14a0a17d'
  Running command git fetch -q https://github.com/unslothai/unsloth.git 31b667b54139962832ea2de890383eed14a0a17d
  Running command git checkout -q 31b667b54139962832ea2de890383eed14a0a17d
  Resolved https://github.com/unslothai/unsloth.git to commit 31b667b54139962832ea2de890383eed14a0a17d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.1 MB

In [21]:
import unsloth
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory


# Load the model and tokenizer from Hugging Face
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.10.10: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [17]:
test_prompt = """Write an improved dataset description for the dataset titled 2019 For Hire Vehicles Trip Data with the category Transportation, and by the agency Taxi and Limousine Commission (TLC). The tags are ['taxi', 'trip data', 'fhv', 'trip', 'base', 'high volume', 'uber', 'lyft', 'via']. The current description is "These records are generated from the For-Hire Vehicle (“FHV”) Trip Record submissions made by traditional livery, luxury, and black car bases. The FHV trip records include fields capturing the dispatching base license number and the pick-up date, time, and taxi zone location ID, which correspond with the NYC Taxi Zones open dataset. Each row represents a single trip in an FHV.".Example row: {
  "dispatching_base_num": "B01239",
  "pickup_datetime": "2019-01-01T00:10:37.000",
  "dropoff_datetime": "2019-01-01T00:26:19.000",
  "dolocationid": "265"
}
Name of the column exactly as it appears in the dataset.: A brief, plain-language explanation of what the data in the column means.
dispatching_base_num: The TLC Base License Number of the base that dispatched the trip
pickup_datetime: The date and time of the trip pick-up
dropOff_datetime: The date and time of the trip dropoff
PUlocationID: TLC Taxi Zone in which the trip began
DOlocationID: TLC Taxi Zone in which the trip ended
SR_Flag: Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line). For shared trips, the value is 1. For non-shared rides, this field is null.
Affiliated_base_number: Base number of the base with which the vehicle is affiliated. This must be provided even if the affiliated base is the same as the dispatching base

Improved dataset description:"""

test_prompt_2 = """You are a data documentation expert.
Write an improved, professional dataset description suitable for an open data catalog.

Dataset title: 2019 For Hire Vehicles Trip Data
Category: Transportation
Agency: Taxi and Limousine Commission (TLC)
Tags: ['taxi', 'trip data', 'fhv', 'trip', 'base', 'high volume', 'uber', 'lyft', 'via']

Current description:
"These records are generated from the For-Hire Vehicle (“FHV”) Trip Record submissions made by traditional livery, luxury, and black car bases. The FHV trip records include fields capturing the dispatching base license number and the pick-up date, time, and taxi zone location ID, which correspond with the NYC Taxi Zones open dataset. Each row represents a single trip in an FHV."

Example row:
{
  "dispatching_base_num": "B01239",
  "pickup_datetime": "2019-01-01T00:10:37.000",
  "dropoff_datetime": "2019-01-01T00:26:19.000",
  "dolocationid": "265"
}

Column definitions:
dispatching_base_num: The TLC Base License Number of the base that dispatched the trip
pickup_datetime: The date and time of the trip pick-up
dropOff_datetime: The date and time of the trip dropoff
PUlocationID: TLC Taxi Zone in which the trip began
DOlocationID: TLC Taxi Zone in which the trip ended
SR_Flag: Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line).
Affiliated_base_number: Base number of the base with which the vehicle is affiliated.

When improving the description:
- Do NOT restate or list individual column definitions.
- Clearly explain what the dataset *represents*, who collects it, and how it can be used.
- Mention the time period, scope (citywide, all FHVs, etc.), and potential analyses (e.g., mobility trends, ride-sharing patterns, policy insights).
- Use natural, informative language rather than bullet points or repetition.
- Keep it concise (1–2 paragraphs).

**Improved description:**
"""


test_prompt_3 = """You are a data documentation expert.
Your task is to rewrite the dataset description so it sounds professional, informative, and engaging — suitable for the NYC Open Data catalog.

Dataset title: 2019 For Hire Vehicles Trip Data
Category: Transportation
Agency: Taxi and Limousine Commission (TLC)
Tags: ['taxi', 'trip data', 'fhv', 'trip', 'base', 'high volume', 'uber', 'lyft', 'via']

Current description:
"These records are generated from the For-Hire Vehicle (“FHV”) Trip Record submissions made by traditional livery, luxury, and black car bases. The FHV trip records include fields capturing the dispatching base license number and the pick-up date, time, and taxi zone location ID, which correspond with the NYC Taxi Zones open dataset. Each row represents a single trip in an FHV."

Example row:
{
  "dispatching_base_num": "B01239",
  "pickup_datetime": "2019-01-01T00:10:37.000",
  "dropoff_datetime": "2019-01-01T00:26:19.000",
  "dolocationid": "265"
}

Column definitions:
dispatching_base_num: The TLC Base License Number of the base that dispatched the trip
pickup_datetime: The date and time of the trip pick-up
dropOff_datetime: The date and time of the trip dropoff
PUlocationID: TLC Taxi Zone in which the trip began
DOlocationID: TLC Taxi Zone in which the trip ended
SR_Flag: Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line).
Affiliated_base_number: Base number of the base with which the vehicle is affiliated.

When improving the description:
- Do NOT restate or list individual column definitions.
- Expand on what the dataset enables — such as transportation planning, ride-share regulation, equity analysis, or urban mobility research.
- Include *context* (why this data matters, who uses it, what insights it offers).
- Use confident, clear, natural language.
- Keep it concise (1–2 paragraphs).
- Write as if it were the official NYC Open Data description.

**Improved description:**
"""

test_prompt_4 = """You are a data documentation expert.
Your task is to rewrite the dataset description so it sounds professional, informative, and engaging — suitable for the NYC Open Data catalog.

Dataset title: 2019 For Hire Vehicles Trip Data
Category: Transportation
Agency: Taxi and Limousine Commission (TLC)
Tags: ['taxi', 'trip data', 'fhv', 'trip', 'base', 'high volume', 'uber', 'lyft', 'via']

Current description:
"These records are generated from the For-Hire Vehicle (“FHV”) Trip Record submissions made by traditional livery, luxury, and black car bases. The FHV trip records include fields capturing the dispatching base license number and the pick-up date, time, and taxi zone location ID, which correspond with the NYC Taxi Zones open dataset. Each row represents a single trip in an FHV."

Example row:
{
  "dispatching_base_num": "B01239",
  "pickup_datetime": "2019-01-01T00:10:37.000",
  "dropoff_datetime": "2019-01-01T00:26:19.000",
  "dolocationid": "265"
}

Column definitions:
dispatching_base_num: The TLC Base License Number of the base that dispatched the trip
pickup_datetime: The date and time of the trip pick-up
dropOff_datetime: The date and time of the trip dropoff
PUlocationID: TLC Taxi Zone in which the trip began
DOlocationID: TLC Taxi Zone in which the trip ended
SR_Flag: Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line).
Affiliated_base_number: Base number of the base with which the vehicle is affiliated.

When improving the description:
- Keep it concise: 1–2 paragraphs maximum.
- Do NOT list or restate individual column definitions.
- Mention what the dataset contains, who collects it, and why it is useful.
- Include potential use cases only once — do not repeat them.
- Write as if it were the official NYC Open Data description.

**Improved description:**
"""

In [24]:
import torch
import pandas as pd
from tqdm import tqdm

# Prepare model for inference
FastLanguageModel.for_inference(model)

# Get token IDs for stopping
eos_id = tokenizer.eos_token_id
eot_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

# Tokenize the test prompt
inputs = tokenizer(test_prompt_4, return_tensors="pt").to("cuda")

# Generate with proper stopping
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=200,
        do_sample=False,
        temperature=0.0,
        num_beams=1,
        eos_token_id=[eos_id, eot_id],
        pad_token_id=eos_id,
        use_cache=True,
    )

# Decode and parse the entire output
response_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

print(response_text)

You are a data documentation expert. 
Your task is to rewrite the dataset description so it sounds professional, informative, and engaging — suitable for the NYC Open Data catalog.

Dataset title: 2019 For Hire Vehicles Trip Data  
Category: Transportation  
Agency: Taxi and Limousine Commission (TLC)  
Tags: ['taxi', 'trip data', 'fhv', 'trip', 'base', 'high volume', 'uber', 'lyft', 'via']  

Current description:
"These records are generated from the For-Hire Vehicle (“FHV”) Trip Record submissions made by traditional livery, luxury, and black car bases. The FHV trip records include fields capturing the dispatching base license number and the pick-up date, time, and taxi zone location ID, which correspond with the NYC Taxi Zones open dataset. Each row represents a single trip in an FHV."

Example row:
{
  "dispatching_base_num": "B01239",
  "pickup_datetime": "2019-01-01T00:10:37.000",
  "dropoff_datetime": "2019-01-01T00:26:19.000",
  "dolocationid": "265"
}

Column definitions:
disp

In [19]:
!pip install openai

In [23]:
from openai import OpenAI

client = OpenAI(
  api_key=KEY
)

response = client.responses.create(
  model="gpt-5-nano",
  input=test_prompt_4,
  store=True,
)

print(response.output_text);


The 2019 For Hire Vehicles Trip Data comprises trip-level records submitted to the NYC Taxi and Limousine Commission (TLC) by registered for-hire vehicle bases, including traditional livery, luxury, and black-car services. Each record represents a single trip and includes key details such as the dispatching base license number, pickup and drop-off times, and zone identifiers that align with the NYC Taxi Zones dataset.

Collected and maintained by the TLC, this data supports transportation planning, policy analysis, and research on trip demand, base activity, and service coverage. The alignment with the Taxi Zones geography enables spatial and temporal analysis at the neighborhood level and informs understanding of high-volume FHV programs and overall for-hire vehicle activity in New York City.
